In [21]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install scikit-learn
# %pip install lightgbm
# %pip install xgboost
# %pip install flask
# %pip install xlrd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import pickle
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [22]:
if not os.path.isdir("templates"):
    os.mkdir("templates")

if not os.path.isdir("models"):
    os.mkdir("models")

In [23]:
details = pd.read_csv("garmin_edge_820/4557226804_ACTIVITY_data.csv", sep=";")
# Pulizia dei dati
details = details.drop(['left_power_phase[degrees]',
                        'left_power_phase_peak[degrees]',
                        'right_power_phase[degrees]',
                        'right_power_phase_peak[degrees]',
                        'left_right_balance'], axis=1)
# convertire i valori di tempo in formato datetime
details['time'] = pd.to_datetime(details.pop('timestamp[s]'), unit='s').dt.time
details.set_index("time", inplace=True)

# calcolo il tempo trascorso dall'inizio dell'attività
details['time_since_start'] = 1
details['time_since_start'] = details['time_since_start'].cumsum().sub(1)

# calcolo le zone di frequenza cardiaca e di potenza dato il battito in input

hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165),
            (166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218),
               (219, 247), (248, 287), (288, float('inf'))]

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone

details['hr_zone'] = details['heart_rate[bpm]'].apply(get_zone, zones=hr_zones)
details['pwr_zone'] = details['power[watts]'].apply(get_zone, zones=power_zones)

# Calcola la differenza di altitudine tra le righe adiacenti
details['altitude_diff'] = details['altitude[m]'] - \
    details['altitude[m]'].shift(1)
details['distance_diff'] = details['distance[m]'] - \
    details['distance[m]'].shift(1)
details[['altitude_diff', 'distance_diff']] = details[[
    'altitude_diff', 'distance_diff']].fillna(0)
# Calcola la percentuale di pendenza
details['slope_percent'] = np.where(
    details['distance_diff'] == 0, 0, details['altitude_diff'] / details['distance_diff'] * 100)
X = details[["distance[m]", "time_since_start"]]
y = details["heart_rate[bpm]"]
model = Pipeline([
    ("scaler", StandardScaler()),
    ("model",  LinearRegression())
])
model.fit(X, y)
with open("models/model.bin", "wb") as f:
    pickle.dump(model, f)

In [24]:
details.describe()

,distance[m],accumulated_power[watts],altitude[m],speed[m/s],power[watts],heart_rate[bpm],cadence[rpm],temperature[C],left_pco[mm],right_pco[mm],time_since_start,hr_zone,pwr_zone,altitude_diff,distance_diff,slope_percent
count,20070.000000,2.007000e+04,20070.000000,20069.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000,20070.000000
mean,81067.030709,1.589602e+06,416.145152,8.460814,149.099402,128.176582,80.905481,12.818635,-3.166716,-1.323916,10034.500000,2.512706,2.688191,0.000488,8.468746,0.991554
std,47347.627841,9.016209e+05,338.943911,2.975603,100.223304,39.738107,24.887026,3.568802,5.395191,4.188786,5793.854287,1.485698,2.104082,0.305215,3.097718,4.230280
min,55.440000,5.870000e+02,23.200000,0.000000,0.000000,30.000000,0.000000,5.000000,-67.000000,-84.000000,0.000000,1.000000,1.000000,-2.600000,0.000000,-38.461538
25%,43580.805000,7.787430e+05,101.200000,6.610000,67.000000,111.000000,77.000000,10.000000,-6.000000,-3.000000,5017.250000,1.000000,1.000000,0.000000,6.620000,0.000000
50%,77644.450000,1.747294e+06,333.200000,8.850000,157.000000,142.000000,90.000000,14.000000,-4.000000,-2.000000,10034.500000,2.000000,1.000000,0.000000,8.840000,0.000000
75%,119649.230000,2.358964e+06,731.600000,10.430000,219.000000,157.000000,95.000000,15.000000,-1.000000,0.000000,15051.750000,4.000000,5.000000,0.200000,10.440000,2.797203
max,170023.180000,3.002034e+06,1113.000000,18.090000,789.000000,185.000000,142.000000,24.000000,123.000000,100.000000,20069.000000,7.000000,7.000000,1.600000,92.160000,38.461538


In [25]:
details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20070 entries, 07:43:58 to 14:07:51
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   distance[m]               20070 non-null  float64
 1   accumulated_power[watts]  20070 non-null  int64  
 2   altitude[m]               20070 non-null  float64
 3   speed[m/s]                20069 non-null  float64
 4   power[watts]              20070 non-null  int64  
 5   heart_rate[bpm]           20070 non-null  int64  
 6   cadence[rpm]              20070 non-null  int64  
 7   temperature[C]            20070 non-null  int64  
 8   left_pco[mm]              20070 non-null  int64  
 9   right_pco[mm]             20070 non-null  int64  
 10  time_since_start          20070 non-null  int64  
 11  hr_zone                   20070 non-null  int64  
 12  pwr_zone                  20070 non-null  int64  
 13  altitude_diff             20070 non-null  float64
 14  d

In [26]:
%%writefile flsk.py
import os.path
import pickle
from flask import Flask, request, render_template

Overwriting flsk.py


In [27]:
%%writefile -a flsk.py

app = Flask(__name__)
app.debug = True

Appending to flsk.py


In [28]:
%%writefile -a  flsk.py

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "GET":
        return render_template("index.html")
    elif request.method == "POST":
        inputs = [
            float(request.form["distance[m]"]),
            float(request.form["altitude[m]"]),
        ]
        with app.open_resource("models/model.bin", "rb") as f:
            model = pickle.load(f)
        response = model.predict([inputs])[0]
        return render_template("index.html", resp=response)

Appending to flsk.py


In [29]:
%%writefile -a  flsk.py

if __name__ == '__main__':
  app.run()

Appending to flsk.py


In [30]:
%%writefile templates/index.html
<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Bootstrap demo</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet"
    integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" crossorigin="anonymous">
</head>

<body class="d-flex flex-column vh-100" data-bs-theme="dark">
  <div class="card m-auto">
    <div class="card-body">
      <h5 class="card-title">Previsione battiti</h5>
      <form method="POST" action="">
        <div class="mb-3">
          <label for="exampleInputEmail1" class="form-label">distance[m]]</label>
          <input name="distance[m]" class="form-control" id="exampleInputEmail1">
        </div>
        <div class="mb-3">
          <label for="exampleInputPassword1" class="form-label">tempo[s]</label>
          <input name="altitude[m]" class="form-control" id="exampleInputPassword1">
        </div>
        <button type="submit" class="btn btn-primary">Submit</button>
      </form>
    </div>
    <div class="card-footer">
      <p>Risultato: <b>{{ resp }}</b></p>
    </div>
  </div>

</body>

</html>

Overwriting templates/index.html
